In [1]:
from Pattern_Analysis import initial_data

80
80


the following files were analyzed: ['Slurm Trials/slurm-9199622_1.out', 'Slurm Trials/slurm-9199622_2.out', 'Slurm Trials/slurm-9199622_3.out', 'Slurm Trials/slurm-9199622_4.out', 'Slurm Trials/slurm-9199622_5.out', 'Slurm Trials/slurm-9199622_6.out', 'Slurm Trials/slurm-9199622_7.out', 'Slurm Trials/slurm-9199622_8.out', 'Slurm Trials/slurm-9199622_9.out', 'Slurm Trials/slurm-9199622_10.out', 'Slurm Trials/slurm-9199622_11.out', 'Slurm Trials/slurm-9199622_12.out', 'Slurm Trials/slurm-9199622_13.out', 'Slurm Trials/slurm-9199622_14.out', 'Slurm Trials/slurm-9199622_15.out', 'Slurm Trials/slurm-9199622_16.out', 'Slurm Trials/slurm-9199622_17.out', 'Slurm Trials/slurm-9199622_18.out', 'Slurm Trials/slurm-9199622_19.out', 'Slurm Trials/slurm-9199622_20.out', 'Slurm Trials/slurm-9199622_21.out', 'Slurm Trials/slurm-9199622_22.out', 'Slurm Trials/slurm-9199622_23.out', 'Slurm Trials/slurm-9199622_24.out', 'Slurm Trials/slurm-9199622_25.out', 'Slurm Trials/slurm-9199622_26.out', 'Slurm Tria

In [6]:
for i in initial_data['index_list']: 
    if len(i) != 474: 
        print(len(i))

In [32]:
import numpy as np
a = initial_data['index_list']
b = np.array([])
for i, j in enumerate(a): 
    b = np.concatenate((b, np.array(j)))
b.resize((80, 474))
b = b.transpose()

In [34]:
print(len(set(b[0])))
print(len(b[0]))
print(len(b))
print(b[0])

6
80
474
[234. 234. 114. 234. 234.   4. 170. 234. 234. 170. 234. 131. 234. 234.
 170. 234. 234. 234. 234. 234. 114. 234. 234. 234. 234. 114. 234. 234.
 234. 131. 234. 234. 131. 234. 234. 170. 114. 234. 234. 234. 234. 170.
 234. 234. 234. 234. 234. 234. 170. 234. 234. 114. 234. 234. 234. 234.
 131. 170. 234. 234. 234. 234. 234. 234. 234. 116. 234. 234. 234. 234.
 234. 116. 234. 234. 170. 114. 234. 170. 234. 234.]
